In [ ]:
seed=101
records_count=10000

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from keras.applications.resnet import ResNet50
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from keras.models import Sequential
from sklearn.metrics import roc_auc_score
from keras import callbacks 
from keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from keras import Model
from keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#download dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nadakhater","key":"e0cabe6cb91e5c18df198be8415307f1"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                      9KB  2022-10-06 08:55:25           5155        171  1.0              
vittoriogiatti/bigmacprice                                      Bigmac Prices                                        14KB  2022-10-19 21:11:14           1133         32  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset               11KB  2022-09-30 04:05:11           4264        122  1.0              
thedevastator/fast-food-restaurants-in-the-united-states    

In [ ]:
! kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100% 5.19G/5.20G [00:45<00:00, 164MB/s]
100% 5.20G/5.20G [00:45<00:00, 124MB/s]


In [ ]:
from zipfile import ZipFile

file_name="/content/skin-cancer-mnist-ham10000.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
	# printing all the contents of the zip file
	# zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

Extracting all the files now...
Done!


In [ ]:
data_pd = pd.read_csv('/content/HAM10000_metadata.csv')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
import os
os.mkdir('HAM10000')

In [ ]:
source_dir="/content/HAM10000_images_part_1"
file_names = os.listdir(source_dir)
for file_name in file_names:
    # print(os.path.join(source_dir, file_name))
    shutil.move(os.path.join(source_dir, file_name), "/content/HAM10000")

source_dir="/content/HAM10000_images_part_2"
file_names = os.listdir(source_dir)
for file_name in file_names:
    # print(os.path.join(source_dir, file_name))
    shutil.move(os.path.join(source_dir, file_name), "/content/HAM10000")


# read meta

In [ ]:
metadata_path="/content/HAM10000_metadata.csv"
meta = pd.read_csv(metadata_path)
meta=meta[:records_count]
images = meta['image_id']
target = meta['dx']

In [ ]:
meta['sex']=meta['sex'].map({'female': 2, 'male': 1})
meta['sex']=meta['sex'].fillna(0)

In [ ]:
meta['dx_type'].count()

10000

In [ ]:
meta['age'].count()

9943

In [ ]:
meta['age']=meta['age'].fillna(-1)

In [ ]:
meta['age'].count()

10000

In [ ]:
will_be_hot=[]
for i in meta.columns:
  if meta[i].dtypes=='O':
    print(i)
    will_be_hot.append(i)

lesion_id
image_id
dx
dx_type
localization


#pre-processing

In [ ]:
for hot in will_be_hot:
  if hot=="image_id" :
    continue
  ls=meta[hot].values.tolist()
  le = preprocessing.LabelEncoder()
  meta[hot] = le.fit_transform(meta[hot]) 

In [ ]:
# #getting features from images
from keras.applications.efficientnet import EfficientNetB4
import random as rn
import tensorflow as tf
import numpy as np
from keras import backend as K

from keras import applications
from keras.applications.efficientnet import EfficientNetB4
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# seed=1   
num_cores=1  
os.environ['PYTHONHASHSEED'] = '0'                      
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                              inter_op_parallelism_threads=num_cores, 
                              allow_soft_placement=True,
                              device_count = {'CPU' : 1, 'GPU' : 0})
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

#getting features from images
model = tf.keras.applications.MobileNetV2(input_shape=(380, 380, 3), include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
from tqdm import tqdm
for indx,i in enumerate(tqdm(meta['image_id'][:5000])):
    path=os.path.join("/content/HAM10000/",i)
    image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
    input_arr=np.array([input_arr])
    input_arr=input_arr/255
    features=model.predict(input_arr)
    nx, ny,_ = features[0].shape
    d2_train_dataset = features[0].reshape(nx*ny*_)
    meta['image_id'][indx]=d2_train_dataset

In [ ]:
from tqdm import tqdm
for indx,i in enumerate(tqdm(meta['image_id'][5000:])):
    path=os.path.join("/content/HAM10000/",i)
    image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
    input_arr=np.array([input_arr])
    input_arr=input_arr/255
    features=model.predict(input_arr)
    nx, ny,_ = features[0].shape
    d2_train_dataset = features[0].reshape(nx*ny*_)
    meta['image_id'][indx]=d2_train_dataset

In [ ]:
type(i)==int

False

In [ ]:
for indx,i in enumerate(tqdm(meta['image_id'])):
    if type(i)==str:
      path=os.path.join("/content/HAM10000/",i)
      image=tf.keras.utils.load_img(path+'.jpg',target_size=(380,380))
      input_arr = tf.keras.preprocessing.image.img_to_array(image)
      input_arr = cv2.cvtColor(input_arr,cv2.COLOR_BGR2RGB)
      input_arr=np.array([input_arr])
      input_arr=input_arr/255
      features=model.predict(input_arr)
      nx, ny,_ = features[0].shape
      d2_train_dataset = features[0].reshape(nx*ny*_)
      meta['image_id'][indx]=d2_train_dataset
      

#split data

In [ ]:
train,test0=train_test_split(meta,test_size=0.2,random_state=seed)
valid,test=train_test_split(test0,test_size=0.5,random_state=seed)

In [ ]:
train['image_id'].shape

(8000,)

In [ ]:
meta[:records_count]['dx'].unique()

array([2, 5, 3, 4, 6, 1, 0])

In [ ]:
meta['dx'].unique()

array([2, 5, 3, 4, 6, 1, 0])

In [ ]:
imgF=[]
for indx in train.index:
  imgF.append(train['image_id'][indx])
imgF_tst=[]
for indx in test.index:
  imgF_tst.append(test['image_id'][indx])

In [ ]:
imgF_valid=[]
for indx in valid.index:
  imgF_valid.append(valid['image_id'][indx])

In [ ]:
target=train['dx']
x=train.drop(['dx','image_id'],axis=1)

target_test=test['dx']
x_test=test.drop(['dx','image_id'],axis=1)

In [ ]:
target_valid=valid['dx']
x_valid=valid.drop(['dx','image_id'],axis=1)

In [ ]:
np.unique(target)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

meta_clf = RandomForestClassifier(random_state=seed)
img_clf = LogisticRegression(random_state=seed,warm_start=True)
img_clf.fit(imgF[1000:2000], target[1000:2000])
img_clf.fit(imgF[2000:3000], target[2000:3000])
img_clf.fit(imgF[3000:4000], target[3000:4000])
img_clf.fit(imgF[4000:5000], target[4000:5000])
img_clf.fit(imgF[5000:6000], target[5000:6000])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=101, warm_start=True)

In [ ]:
img_clf.fit(imgF[:1000], target[:1000])
img_clf.fit(imgF[6000:7000], target[6000:7000])
img_clf.fit(imgF[7000:8000], target[7000:8000])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(random_state=101, warm_start=True)

In [ ]:
meta_clf.fit(x, target)

RandomForestClassifier(random_state=101)

In [ ]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

def multiclass_roc_auc_score(y_test, y_pred, average="weighted"):
        lb = LabelBinarizer()
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)

        return  metrics.accuracy_score(y_test,y_pred)
        # return metrics.roc_auc_score(y_test, y_pred, average=average)

#meta + image in one model

In [ ]:
predicted_img_labels_tst = img_clf.predict_proba(imgF_tst)
predicted_meta_labels_tst = meta_clf.predict_proba(x_test)

In [ ]:
predicted_img_labels_valid = img_clf.predict_proba(imgF_valid)
predicted_meta_labels_valid = meta_clf.predict_proba(x_valid)

In [ ]:
# pre_img=[]
# for i in predicted_img_labels_valid:
#   pre_img.append(np.argmax(i))
# pre_meta=[]
# for i in predicted_meta_labels_valid:
#   pre_meta.append(np.argmax(i))

pre_img=[]
for i in predicted_img_labels_tst:
  pre_img.append(np.argmax(i))
pre_meta=[]
for i in predicted_meta_labels_tst:
  pre_meta.append(np.argmax(i))

In [ ]:
print('img',multiclass_roc_auc_score(pre_img,target_test)*100)
print('meta',multiclass_roc_auc_score(pre_meta,target_test)*100)

img 48.699999999999996
meta 87.0


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(pre_img,target_test, average='weighted')*100)
print("Recall_score: ",recall_score(pre_img,target_test, average='weighted')*100)
print("F1_score: ",f1_score(pre_img,target_test, average='weighted')*100)

Precision_score:  51.798240496717526
Recall_score:  48.699999999999996
F1_score:  50.09122344037632


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(pre_meta,target_test, average='weighted')*100)
print("Recall_score: ",recall_score(pre_meta,target_test, average='weighted')*100)
print("F1_score: ",f1_score(pre_meta,target_test, average='weighted')*100)

Precision_score:  87.57942643751657
Recall_score:  87.0
F1_score:  87.17564540382831


In [ ]:
all_x=[]
all_im=[]
all_y=[]
for i in range(len(predicted_meta_labels_valid)):
  all_x.append(predicted_meta_labels_valid[i])
  all_im.append(predicted_img_labels_valid[i])
  all_y.append(target_valid.iloc[i])

  
  all_x.append(predicted_meta_labels_tst[i])
  all_im.append(predicted_img_labels_tst[i])
  all_y.append(target_test.iloc[i])

In [ ]:
# df = pd.DataFrame(lst)
dict = {'meta': all_x, 'image': all_im, 'label': all_y} 
df = pd.DataFrame(dict)

In [ ]:
df

,meta,image,label
0,"[0.01, 0.03, 0.0, 0.0, 0.3, 0.66, 0.0]","[8.672333626821304e-12, 4.134649027206074e-14,...",5
1,"[0.0, 0.0, 0.0, 0.01, 0.3, 0.69, 0.0]","[3.336301682963103e-17, 3.200665658747231e-10,...",5
2,"[0.0, 0.01, 0.0, 0.0, 0.01, 0.98, 0.0]","[5.164703419762506e-10, 9.157586944524229e-06,...",5
3,"[0.0, 0.34, 0.65, 0.0, 0.01, 0.0, 0.0]","[4.2013930856785045e-17, 1.5840299656967075e-2...",2
4,"[0.0, 0.0, 0.9, 0.0, 0.03, 0.07, 0.0]","[5.617496013267208e-17, 1.0231953826053212e-09...",2
...,...,...,...
1995,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[6.358985908930893e-11, 0.07964209652581931, 0...",5
1996,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[1.0226580195083473e-10, 4.441811155761645e-09...",5
1997,"[0.0, 0.0, 0.0, 0.0, 0.04, 0.96, 0.0]","[8.29443100928913e-14, 0.006884381145093782, 8...",5
1998,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0.10756629059514798, 0.21300263759519306, 8.1...",5


In [ ]:
train,test0=train_test_split(df,test_size=0.2,random_state=seed)
test,valid=train_test_split(test0,test_size=0.5,random_state=seed)

In [ ]:
x_train=train.drop(['label'],axis=1)
y_train=train['label']
x_test=test.drop(['label'],axis=1)
y_test=test['label']
x_valid=valid.drop(['label'],axis=1)
y_valid=valid['label']

In [ ]:
np.array(x_train['meta'].tolist())

array([[0.16, 0.01, 0.17, ..., 0.12, 0.54, 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.85, ..., 0.  , 0.09, 0.05],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.04, 0.  , 0.37, ..., 0.47, 0.11, 0.  ]])

In [ ]:
y_train

668     5
1345    5
373     2
1388    2
132     4
       ..
1599    5
1862    2
1361    5
1547    5
863     2
Name: label, Length: 1600, dtype: int64

In [ ]:
np.argmax(np.array(x_train['image'].tolist()[2]))

5

In [ ]:
pr_img=x_train['image'].tolist()
pr_meta=x_train['meta'].tolist()
both=0
only_img=0
only_meta=0
print(len(pr_img))
for indx,i in enumerate(y_train):
  # print(pr_meta[0])
  if np.argmax(pr_meta[indx])==i and np.argmax(pr_img[indx])==i:
    both+=1
  elif np.argmax( pr_meta[indx])==i:
    only_meta+=1
  elif np.argmax(pr_img[indx])==i:
    only_img+=1
print('only_img',only_img)
print('only_meta',only_meta)
print('both',both)

1600
only_img 54
only_meta 685
both 697


In [ ]:
import os 
import random as rn
import tensorflow as tf
import numpy as np
from keras import backend as K
# seed=1
num_cores=1  
os.environ['PYTHONHASHSEED'] = '0'                      
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                              inter_op_parallelism_threads=num_cores, 
                              allow_soft_placement=True,
                              device_count = {'CPU' : 1, 'GPU' : 0})
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

from tensorflow import keras
from keras import regularizers
model_inputs=[]
for i in range(2):
  model_inputs.append(keras.Input(shape=(7,)))

combinedInput = tf.keras.layers.concatenate(model_inputs)
# 390
x = Dense(1024,activation="relu",
          kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
          )(combinedInput)

x=Dropout(.9)(x)

predictions = Dense(7,activation="softmax",)(x)
model = Model(inputs=model_inputs,outputs=predictions)

opt= keras.optimizers.Adam(learning_rate=0.0001,epsilon=0.01)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=['accuracy'])

# valid
model.fit([np.array(x_train['meta'].tolist()),np.array(x_train['image'].tolist())],y_train ,epochs=200,validation_data=([np.array(x_valid['meta'].tolist()),np.array(x_valid['image'].tolist())],y_valid),batch_size=16)#validation_data=(validList,validDic['diagnostic'])

print("evaluate")

Epoch 1/200
100/100 [==============================] - 1s 5ms/step - loss: 1.9318 - accuracy: 0.1900 - val_loss: 1.8466 - val_accuracy: 0.5950
Epoch 2/200
100/100 [==============================] - 0s 3ms/step - loss: 1.7835 - accuracy: 0.4756 - val_loss: 1.6915 - val_accuracy: 0.6550
Epoch 3/200
100/100 [==============================] - 0s 3ms/step - loss: 1.6219 - accuracy: 0.6475 - val_loss: 1.5369 - val_accuracy: 0.6550
Epoch 4/200
100/100 [==============================] - 0s 3ms/step - loss: 1.4674 - accuracy: 0.6750 - val_loss: 1.3898 - val_accuracy: 0.6550
Epoch 5/200
100/100 [==============================] - 0s 4ms/step - loss: 1.3244 - accuracy: 0.6737 - val_loss: 1.2634 - val_accuracy: 0.6550
Epoch 6/200
100/100 [==============================] - 0s 3ms/step - loss: 1.2162 - accuracy: 0.6706 - val_loss: 1.1603 - val_accuracy: 0.6550
Epoch 7/200
100/100 [==============================] - 0s 3ms/step - loss: 1.1123 - accuracy: 0.6675 - val_loss: 1.0840 - val_accuracy: 0.6550

In [ ]:
# test
model_loss,modelAccuracy = model.evaluate([np.array(x_test['meta'].tolist()),np.array(x_test['image'].tolist())],y_test)
print("model accuracy "+str(modelAccuracy))

7/7 [==============================] - 0s 3ms/step - loss: 0.3576 - accuracy: 0.9050
model accuracy 0.9049999713897705


In [ ]:
predictions = model.predict([np.array(x_test['meta'].tolist()),np.array(x_test['image'].tolist())], verbose=0)

In [ ]:
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report,roc_auc_score,roc_curve,auc
from tensorflow.keras.utils import to_categorical

#geting predictions on test dataset
y_pred = np.argmax(predictions, axis=1)

#getting the true labels per image 
y_true = y_test

# Creating classification report 
report = classification_report(y_true, y_pred)

print("\nClassification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       0.78      0.70      0.74        10
           2       0.75      0.71      0.73        17
           3       0.00      0.00      0.00         1
           4       0.89      0.83      0.86        29
           5       0.93      0.96      0.95       140

    accuracy                           0.91       200
   macro avg       0.72      0.70      0.71       200
weighted avg       0.90      0.91      0.90       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision_score: ",precision_score(y_true, y_pred, average='weighted')*100)
print("Recall_score: ",recall_score(y_true, y_pred, average='weighted')*100)
print("F1_score: ",f1_score(y_true, y_pred, average='weighted')*100)

Precision_score:  89.82519157088123
Recall_score:  90.5
F1_score:  90.11038961038962


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
